# Start

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pyodbc

from pat_back import PAT_FLAG, VALIDATE_RULE, COVERAGE_TYPE, PERIL_SUBGROUP, DEDDUCT_TYPE, RATING_TYPE, PSOLD_PERSP
from pat_back import PsoldRating, FlsRating

fld = r"C:\Users\cxiao\source\repos\property-exposure\gcpropexpo\GCPropExpo\tests\testthat"
df_loc = pd.read_csv(f'{fld}/Inputs/PATLocations.csv')
df_loc['LOCID'] = df_loc.index +1
df_pol = pd.read_csv(f'{fld}/Inputs/PATPolicies.csv')
df_res = pd.read_csv(f'{fld}/Controls/PAT_Ans.csv')

DT = df_pol.merge(df_loc, on='Policy ID')
DT.columns=['PolicyID', 'Limit', 'Retention', 'PolPrem', 'LossRatio', 'Participation', 'TIV', 'Stack', 'RatingGroup', 'LOCID']



### Input Parameters

In [2]:
# tol<- 1e-5
# masterAns <- fread('./Controls/PAT_Ans.csv')
# uflsNoPar <- 'User-defined FLS curve selected without passing proper parameters.'
# uflsBadPar <- 'User-defined FLS curve selected with incorrect parameter names.'

# test_that("Accuracy", {
#   loc <- fread('./Inputs/PATLocations.csv')
#   pol <- fread('./Inputs/PATPolicies.csv')
#   covgInfo <- list(subPrem = 4e6,
#                    ratio = 0.8,
#                    dedType = 1L,
#                    curveID = 57L,
#                    adCvg = 0,
#                    flsPars = list(tau = 0.169748277,
#                                   mu = 0.000639849,
#                                   w = 0.678145419,
#                                   cap = 1,
#                                   theta = 1.27537e-05,
#                                   beta = 0.169748277),
#                    type = 'fls')
#   correctAns <- masterAns[TEST == 'FLS1', .(LOCID, PREMIUM)]
#   testResults <- PAT(pol, loc, covgInfo)
#   expect_equal(testResults$allocPrem, correctAns$PREMIUM, tolerance = tol)
# })

ratio = 0.8
covg = COVERAGE_TYPE['Building_Contents_BI']
adCvg = 0
dedType = DEDDUCT_TYPE['Retains_Limit']
curveID = 57
para={
    'w' : 0.678145419,
    'tau' : 0.169748277,
    'mu' : 0.000639849,
    'theta' : 1.27537e-05,
    'cap' : 1,
    'beta' : 0.169748277,
    'something_extra': 1234
}
test = 'FLS1'

DT.fillna({'LossRatio': ratio}, inplace=True)

### Curve data

In [3]:
conn_str = f'''DRIVER={{SQL Server}};Server=dfwcat-pm;Database=premium_allocation_tool;
            User Id=sa;Password=In5tr@t1;MultipleActiveResultSets=true;'''
with pyodbc.connect(conn_str) as conn:
     df_fls = pd.read_sql_query(f"""select ID, mu, w, tau, theta, beta, cap, uTgammaMean, limMean
                from fls_curves order by ID""", conn).set_index('ID')

try: para
except:
     pass
else: 
     df_fls.loc[curveID] = para

### Allocation

In [4]:
rating_model = FlsRating(df_fls, curve_id = curveID, **para)
DT = rating_model.calculate_las(DT, addt_cvg = adCvg, ded_type = dedType.name)

DT['Premium'] = (DT.PolLAS-DT.DedLAS) * DT.LossRatio 
sumLAS = DT.groupby('PolicyID').agg( 
    {'Premium': 'sum'}).rename(columns={'Premium': 'sumLAS'})
DT = DT.merge(sumLAS, on='PolicyID')
DT['Premium'] *= DT['PolPrem'] / DT['sumLAS']
DT.fillna({'Premium':0},inplace=True)


### Compare

In [5]:
df = df_res[df_res.TEST== test].merge(DT, on ='LOCID')[['LOCID','PREMIUM','Premium']]
df['Diff']= np.abs(df.PREMIUM - df.Premium)/df.PREMIUM
print('Accuracy: ', df.Diff.max())
df

Accuracy:  1.6562787135819653e-06


,LOCID,PREMIUM,Premium,Diff
0,1,2.647400e+05,2.647400e+05,3.731873e-08
1,2,1.486120e+05,1.486121e+05,8.006990e-07
2,3,9.653534e+05,9.653534e+05,3.531283e-08
3,4,0.000000e+00,0.000000e+00,NaN
4,5,9.176738e+03,9.176744e+03,6.516172e-07
5,6,3.000000e+06,3.000000e+06,0.000000e+00
6,7,6.679313e+06,6.679313e+06,1.024124e-07
7,8,8.467144e+04,8.467130e+04,1.656279e-06
8,9,0.000000e+00,0.000000e+00,NaN
9,10,0.000000e+00,0.000000e+00,NaN
